In [36]:
import requests
from bs4 import BeautifulSoup,Comment
import pandas as pd
import time
import os
import sys
import unicodedata
import re
# URL of the NBA awards page
import numpy as np
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.static import players,teams

# Get current season year in the format "2023-24" for example

# URL of the NBA awards page
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

ps = True
if ps:
    trail='_ps'
else:
    trail=''
def pull_bref(ps=False,totals=False):
    leagues = "playoffs" if ps else "leagues"
    frames = []
    for year in range(2025, 2026):
        if totals ==True:
            url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_totals.html"
            pt_index=28
        else:
            url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_per_poss.html"
            pt_index=28
        print(url)
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8' 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table
        table = soup.find('table')
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        
        # Define the data structure to store extracted rows
        data = []
        for row in rows:
            cells = row.find_all('td')
            if cells:
                # Extract player name, url, team, and stats required
                player_cell = cells[0]
                player_name = player_cell.text if player_cell.text else "N/A"  # Player name
                player_url = "https://www.basketball-reference.com" + player_cell.a['href'] if player_cell.a else "N/A"  # Player URL
                team_acronym = cells[2].text if cells[2].text else "N/A"  # Team acronym
                
                gp = cells[4].text if len(cells) > 4 else "0"  # Minutes played
                mp = cells[6].text if len(cells) > 6 else "0"  # Minutes played

                # Columns required for True Shooting Percentage calculation
                fga = cells[8].text if len(cells) > 8 else "0"  # Field Goal Attempts
                fg = cells[7].text if len(cells) > 7 else "0"   # Field Goals Made
                tpa = cells[11].text if len(cells) > 11 else "0" # Three-Point Attempts
                tp = cells[10].text if len(cells) > 10 else "0"  # Three-Point Made
                fta = cells[18].text if len(cells) > 18 else "0" # Free Throw Attempts
                ft = cells[17].text if len(cells) > 17 else "0"  # Free Throws Made
                pts = cells[pt_index].text if len(cells) > pt_index else "0"  #
                
                data.append([
                    player_name, player_url, team_acronym, year, gp,mp, fga, fg, tpa, tp, fta, ft,pts
                ])
        
        # Create DataFrame for the current year
        year_data = pd.DataFrame(
            data=data, 
            columns=['player', 'url', 'team', 'year', 'G','MP', 'FGA', 'FG', '3PA', '3P', 'FTA', 'FT','PTS']
        )
        frames.append(year_data)
        print(f"Year {year} data added.")
        time.sleep(2)
    
    return pd.concat(frames)


index_frame=pull_bref(ps=ps,totals=True)
index_frame['bref_id']=index_frame['url'].str.split('/',expand=True)[5]
index_frame['bref_id']=index_frame['bref_id'].str.split('.',expand=True)[0]
master = pd.read_csv('index_master'+trail+'.csv')
match_dict=dict(zip(master['bref_id'],master['nba_id']))

team_dict=dict(zip(master['team'],master['team_id']))

search_dict={
    "hollaro01": 1641842,
    "sarral01": 1642259,
    "dadiepa01": 1642359,
    "cuiyo01": 1642385,
    "dasiltr01": 1641783,
    "shannte01":1630545

}
match_dict.update(search_dict)
index_frame['nba_id']=index_frame['bref_id'].map(match_dict)



current_season = "2024-25"  # Update this to the current season if needed

# Fetch all players for the current season
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, season=current_season)
players_list = players_data.get_data_frames()[0]

# Display a list of player names
player_names = dict(zip(players_list['DISPLAY_FIRST_LAST'],players_list['PERSON_ID']))

notfound=index_frame[index_frame.year==2025].reset_index(drop=True)
notfound=index_frame[index_frame.nba_id.isna()].reset_index(drop=True)
print(notfound)
notfound['nba_id']=notfound['player'].map(player_names)
notfound.dropna(inplace=True)
index_frame.dropna(inplace=True)



index_frame=pd.concat([index_frame,notfound])
index_frame['team_id']=index_frame['team'].map(team_dict)
index_copy = index_frame[['player', 'url', 'year', 'team', 'bref_id', 'nba_id', 'team_id']]
master=master[master.year!=2025]
master=pd.concat([master,index_copy])
master.drop_duplicates(inplace=True)
master.to_csv('index_master'+trail+'.csv',index=False)
index_frame.dropna(subset='bref_id',inplace=True)
index_frame['FTA']=index_frame['FTA'].astype(int)
index_frame['FGA']=index_frame['FGA'].astype(int)

index_frame['PTS']=index_frame['PTS'].astype(int)
year=2025
old_scoring=pd.read_csv('totals'+trail+'.csv')
old_scoring=old_scoring[old_scoring.year<year]
old_scoring.columns

index_frame['TS%'] = (index_frame['PTS'] / (2 * (index_frame['FGA'] + 0.44 * index_frame['FTA']))) * 100

# Select and rename columns to match scoring.csv
new_df = index_frame[['player', 'TS%', 'PTS', 'MP', 'team', 'G', 'FTA','FGA','year', 'nba_id']].copy()
new_df = new_df.rename(columns={
    'player': 'Player',
    'team': 'Tm'
})

# Display the resulting DataFrame
new_scoring=pd.concat([old_scoring,new_df])
new_scoring.fillna(0,inplace=True)
new_scoring.replace([np.inf, -np.inf], 0, inplace=True)
new_scoring.loc[new_scoring['TS%'] > 150, 'TS%'] = 0

new_scoring.to_csv('totals'+trail+'.csv',index=False)
new_scoring[new_scoring.nba_id==2544]

https://www.basketball-reference.com/playoffs/NBA_2025_totals.html
Year 2025 data added.
Empty DataFrame
Columns: [player, url, team, year, G, MP, FGA, FG, 3PA, 3P, FTA, FT, PTS, bref_id, nba_id]
Index: []


,Player,TS%,PTS,MP,Tm,G,FTA,FGA,year,nba_id
5428,LeBron James,55.722724,400.0,604.0,CLE,13,118.0,307.0,2006,2544.0
5636,LeBron James,51.623939,501.0,893.0,CLE,20,196.0,399.0,2007,2544.0
5825,LeBron James,52.513774,366.0,552.0,CLE,13,167.0,275.0,2008,2544.0
6017,LeBron James,61.818000,494.0,580.0,CLE,14,199.0,312.0,2009,2544.0
6224,LeBron James,60.652009,320.0,460.0,CLE,11,120.0,211.0,2010,2544.0
6426,LeBron James,56.267548,497.0,922.0,MIA,21,156.0,373.0,2011,2544.0
6631,LeBron James,57.607115,697.0,983.0,MIA,23,234.0,502.0,2012,2544.0
6830,LeBron James,58.546169,596.0,960.0,MIA,23,175.0,432.0,2013,2544.0
7034,LeBron James,66.764133,548.0,763.0,MIA,20,160.0,340.0,2014,2544.0
7236,LeBron James,48.665544,601.0,844.0,CLE,20,167.0,544.0,2015,2544.0


In [37]:

def pull_bref_score(ps=False,totals=False):
    leagues = "playoffs" if ps else "leagues"
    frames = []
    for year in range(2025, 2026):
   
        url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_per_poss.html"

        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        
        response.encoding = 'utf-8' 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table
        table = soup.find('table')
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        
        # Define the data structure to store extracted rows
        data = []
        for row in rows:
            cells = row.find_all('td')
            if cells:
                pt_index=28
                # Extract player name, url, team, and stats required
                player_cell = cells[0]
                player_name = player_cell.text if player_cell.text else "N/A"  # Player name
                player_url = "https://www.basketball-reference.com" + player_cell.a['href'] if player_cell.a else "N/A"  # Player URL
                team_acronym = cells[2].text if cells[2].text else "N/A"  # Team acronym
                
                gp = cells[4].text if len(cells) > 4 else "0"  # Minutes played
                mp = cells[6].text if len(cells) > 6 else "0"  # Minutes played

                # Columns required for True Shooting Percentage calculation
                fga = cells[8].text if len(cells) > 8 else "0"  # Field Goal Attempts
                fg = cells[7].text if len(cells) > 7 else "0"   # Field Goals Made
                tpa = cells[11].text if len(cells) > 11 else "0" # Three-Point Attempts
                tp = cells[10].text if len(cells) > 10 else "0"  # Three-Point Made
                fta = cells[21].text if len(cells) > 21 else "0" # Free Throw Attempts
                ft = cells[20].text if len(cells) > 20 else "0"  # Free Throws Made
                pts = cells[pt_index].text if len(cells) > pt_index else "0"  # Free Throws Made
                
                data.append([
                    player_name, player_url, team_acronym, year, gp,mp, fga, fg, tpa, tp, fta, ft,pts
                ])
        
        # Create DataFrame for the current year
        year_data = pd.DataFrame(
            data=data, 
            columns=['player', 'url', 'team', 'year', 'G','MP', 'FGA', 'FG', '3PA', '3P', 'FTA', 'FT','PTS']
        )
        frames.append(year_data)
        print(year_data.iloc[0])
        print(f"Year {year} data added.")
        time.sleep(2)
    
    return pd.concat(frames)
index_frame=pull_bref_score(ps=ps)
index_frame['bref_id']=index_frame['url'].str.split('/',expand=True)[5]
index_frame['bref_id']=index_frame['bref_id'].str.split('.',expand=True)[0]

match_dict=dict(zip(master['bref_id'],master['nba_id']))

team_dict=dict(zip(master['team'],master['team_id']))

search_dict={
    "hollaro01": 1641842,
    "sarral01": 1642259,
    "dadiepa01": 1642359,
    "cuiyo01": 1642385,
    "dasiltr01": 1641783,
    "salauti01":1642275,
    "shannte01":1630545

}


match_dict.update(search_dict)
index_frame['nba_id']=index_frame['bref_id'].map(match_dict)

current_season = "2024-25"  # Update this to the current season if needed

# Fetch all players for the current season
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, season=current_season)
players_list = players_data.get_data_frames()[0]

# Display a list of player names
player_names = dict(zip(players_list['DISPLAY_FIRST_LAST'],players_list['PERSON_ID']))

notfound=index_frame[index_frame.year==2025].reset_index(drop=True)
notfound=index_frame[index_frame.nba_id.isna()].reset_index(drop=True)
notfound['nba_id']=notfound['player'].map(player_names)
notfound.dropna(inplace=True)
index_frame.dropna(inplace=True)



index_frame=pd.concat([index_frame,notfound])

index_frame['team_id']=index_frame['team'].map(team_dict)

index_frame.dropna(subset='bref_id',inplace=True)
index_frame.fillna(0,inplace=True)
print(index_frame)
index_frame.replace('',0,inplace=True)
index_frame['FTA']=index_frame['FTA'].astype(float)
index_frame['FGA']=index_frame['FGA'].astype(float)

index_frame['PTS']=index_frame['PTS'].astype(float)
year=2025
old_scoring=pd.read_csv('scoring'+trail+'.csv')
old_scoring=old_scoring[old_scoring.year<year]
old_scoring.columns

index_frame['TS%'] = (index_frame['PTS'] / (2 * (index_frame['FGA'] + 0.44 * index_frame['FTA']))) * 100

# Select and rename columns to match scoring.csv
new_df = index_frame[['player', 'TS%', 'PTS', 'MP', 'team', 'G', 'year', 'nba_id']].copy()
new_df = new_df.rename(columns={
    'player': 'Player',
    'team': 'Tm'
})

# Display the resulting DataFrame
new_scoring=pd.concat([old_scoring,new_df])
new_scoring.fillna(0,inplace=True)
new_scoring.loc[new_scoring['TS%'] > 150, 'TS%'] = 0

new_scoring.to_csv('scoring'+trail+'.csv',index=False)

new_scoring=pd.read_csv('scoring.csv')
gp=new_scoring[['nba_id','Player','year','G']].reset_index()
gp.to_csv('../player_sheets/lineups/games.csv',index=False)

ps_scoring=pd.read_csv('scoring_ps.csv')
ps_scoring.fillna(0,inplace=True)
ps_scoring.loc[ps_scoring['TS%'] > 150, 'TS%'] = 0


ps_gp=ps_scoring[['nba_id','Player','year','G']].reset_index()
ps_gp.to_csv('../player_sheets/lineups/ps_games.csv',index=False)
ps_gp.to_csv('../extra_data/wowy_leverage/ps_games.csv',index=False)
ps_gp
ps_scoring


player                             Nickeil Alexander-Walker
url       https://www.basketball-reference.com/players/a...
team                                                     26
year                                                   2025
G                                                         1
MP                                                       20
FGA                                                    24.9
FG                                                      5.5
3PA                                                    16.6
3P                                                      2.8
FTA                                                     2.8
FT                                                      2.8
PTS                                                        
Name: 0, dtype: object
Year 2025 data added.
                      player  \
0   Nickeil Alexander-Walker   
1      Giannis Antetokounmpo   
2                 OG Anunoby   
3              Nicolas Batum   
4              

/tmp/ipykernel_86047/19456763.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  index_frame.replace('',0,inplace=True)


,Player,TS%,PTS,MP,Tm,G,year,nba_id
0,Zaid Abdul-Aziz,51.502980,15.9,2459.0,HOU,79,1974,76002.0
1,Kareem Abdul-Jabbar*,56.254004,28.1,3548.0,MIL,81,1974,76003.0
2,Don Adams,45.660701,14.9,2298.0,DET,74,1974,76012.0
3,Rick Adelman*,44.808816,13.5,618.0,CHI,55,1974,76015.0
4,Lucius Allen,53.568266,24.2,2388.0,MIL,72,1974,76027.0
...,...,...,...,...,...,...,...,...
25828,Gabe Vincent,0.000000,0.0,25.0,28,1,2025,1629216.0
25829,Jarace Walker,0.000000,0.0,17.0,21,1,2025,1641716.0
25830,Peyton Watson,0.000000,0.0,14.0,22,1,2025,1631212.0
25831,Russell Westbrook,0.000000,0.0,34.0,36,1,2025,201566.0


In [38]:
scoring=pd.read_csv('scoring.csv')
scoring[scoring.year==2024]

,Player,TS%,PTS,MP,Tm,G,year,nba_id
25013,Precious Achiuwa,54.416205,17.3,1624.0,TOT,74,2024,1630173.0
25014,Precious Achiuwa,51.142912,21.3,437.0,TOR,25,2024,1630173.0
25015,Precious Achiuwa,56.348074,15.8,1187.0,NYK,49,2024,1630173.0
25016,Bam Adebayo,57.504078,28.2,2416.0,MIA,71,2024,1628389.0
25017,Ochai Agbaji,49.985079,13.4,1641.0,TOT,78,2024,1630534.0
...,...,...,...,...,...,...,...,...
25743,Thaddeus Young,51.622419,12.6,89.0,PHO,10,2024,201152.0
25744,Trae Young,58.444656,34.3,1942.0,ATL,54,2024,1629027.0
25745,Omer Yurtseven,56.609195,19.7,545.0,UTA,48,2024,1630209.0
25746,Cody Zeller,48.124793,11.6,320.0,NOP,43,2024,203469.0
